## Analyzing borrowers’ risk of defaulting

This project is to prepare a report for a bank’s loan division. I need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

This report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [1]:
#read csv file and getting information of the dataset
import pandas as pd
credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
#getting the first 10 row in dataset
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


### Conclusion

The dataset has 12 columns and 21525 entries. There are three datatypes in the dataset: integer, float, and object. There are some missing values in "days_employed" and "total_income" columns. The datatype of "days_employed" needs to be changed as integer. The elements in "education" column need to be changed as lowercase for further use. The similar entries in "purpose" column need to be categorized by unique purpose.

### Step 2. Data preprocessing

### Processing missing values

In [3]:
#changing 'days_employed' to positive
#remove duplicated row in dataset
credit_scoring['days_employed'] = credit_scoring['days_employed'].abs()
credit_scoring = credit_scoring.drop_duplicates()

#filling missing values
credit_scoring['days_employed'] = credit_scoring['days_employed'].fillna(credit_scoring['days_employed'].mean())
credit_scoring['total_income'] = credit_scoring['total_income'].fillna(credit_scoring['total_income'].mean())

### Conclusion

I changed the days_employed to positive. I dropped duplicated rows in dataset. I filled missing values using mean. The possible reason of duplicate is human error.

### Data type replacement

In [4]:
credit_scoring['days_employed'] = credit_scoring['days_employed'].astype('int')

### Conclusion

I changed the 'days_employed' datatype to integer because number of days is always integer.

### Processing duplicates

In [5]:
credit_scoring.drop_duplicates().reset_index(drop=True)
credit_scoring.drop(credit_scoring[credit_scoring['children'] == -1].index, inplace=True)
credit_scoring.drop(credit_scoring[credit_scoring['children'] == 20].index, inplace=True)

### Conclusion

I dropped once more duplicated row in datasets and reset indexing. I dropped rows with 'children' equal to -1 or 20 as they seem as errors and represent just 0.1% for the dataset.

### Categorizing Data 

In [6]:
#change any string entries into lowercase
string_columns = ['education','family_status','income_type','purpose']
for row in string_columns:
    credit_scoring[row] = credit_scoring[row].str.lower()
    
def purpose_grouping(purpose):
    wedding_words = ['wedding']
    real_estate_words = ['real estate','transaction','transactions','property']
    home_words = ['house','housing']
    car_words = ['car','cars']
    school_words = ['education','educated','university']
    
    #check any 'wedding' words in the purpose   
    for wedding_word in wedding_words:
        if wedding_word in purpose:
            return 'wedding'
    
    #check any 'real estate' words in the purpose
    for real_estate_word in real_estate_words:
        if real_estate_word in purpose:
            return 'real estate'
    
    #check any 'home' words in the purpose    
    for home_word in home_words:
        if home_word in purpose:
            return 'house'
        
    #check any 'car' words in the purpose
    for car_word in car_words:
        if car_word in purpose:
            return 'car'
        
    #check any 'school' words in the purpose
    for school_word in school_words:
        if school_word in purpose:
            return 'education'
        

credit_scoring['purpose_type'] = credit_scoring['purpose'].apply(purpose_grouping)

### Conclusion

I changed the entries in 'purpose' column to lowercase in order to facilitate the grouping. I categorized the same type of purpose in the 'purpose' column because there are many purposes which are actually the same. I keep using grouping because I saw which word was not belong to purpose type and it is there now. Now, any purpose belongs to purpose type.

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [7]:
credit_scoring.pivot_table(index=['children'], values=['debt'], aggfunc='mean')

,debt
children,
0,0.075353
1,0.092327
2,0.094542
3,0.081818
4,0.097561
5,0.000000


### Conclusion

The purcentage of people who defaulted on loan: 9.4% with 4 children, 9.4% will 2 children, 9.2% with 1 children, and 7.5% with no child. The more child you have, the more default on paying loan will be.

- Is there a relation between marital status and repaying a loan on time?

In [8]:
credit_scoring.pivot_table(index=['family_status'], values=['debt'], aggfunc='mean')

,debt
family_status,
civil partnership,0.092861
divorced,0.070648
married,0.075575
unmarried,0.097639
widow / widower,0.066246


### Conclusion

The 9.7% of defaulted people are unmarried, 9.2% are civil partnership, 7.5% are married, 7% are devorced, and 6.6% are widow/widower.

- Is there a relation between income level and repaying a loan on time?

In [9]:
def income_level_grouping(total_income):
    if total_income < 50000:
        return 'low'
    if 50001 < total_income < 100000:
        return 'medium'
    if 100001 < total_income:
        return 'high'

credit_scoring['income_level'] = credit_scoring['total_income'].apply(income_level_grouping)

credit_scoring.pivot_table(index=['income_level'], values=['debt'], aggfunc='mean')

,debt
income_level,
high,0.060606
low,0.081869
medium,0.070666


### Conclusion

8.1% of poeple with low income level defaulted on paying loan. 7% of poeple with medium income level defaulted on paying loan. 6% of people with high income level defaulted on paying loan.

- How do different loan purposes affect on-time repayment of the loan?

In [10]:
credit_scoring.pivot_table(index=['purpose_type'], values=['debt'], aggfunc='mean')

,debt
purpose_type,
car,0.093436
education,0.092504
house,0.066221
real estate,0.075135
wedding,0.078744


### Conclusion

9.3% of poeple with car purpose defaulted on loan payment. 9.2% of people defaulted with education purpose. 7.8% of people defaulted with wedding purpose. 7.5% of people defaulted with real estate purpose. 6.6% of poeple defaulted with house purpose.

### Step 4. General conclusion

In this project, many processes steps have been taken, starting from identifying and removing duplicated entries, changing datatypes, filling missing values, grouping, and finaly finding relationships on data. The results showed that: (1) the more children you have the more probable you will be defaulted on paying loan, (2) the unmarried showed the highest purcentage of people who defaulted on loan based on family status, (3) poeple with low income level showed the highest purcentage on defaulted poeple based on income level, and (4) car purpose showed the highest purcentage of defaulted people based on purpose type.

### Project Readiness Checklist

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.